In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install opencv-python-headless
!pip install mtcnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.8 MB/s eta 0:00:00


In [ ]:
import cv2
from mtcnn import MTCNN
from PIL import Image
import pandas as pd

detector = MTCNN()

def extract_frames_and_detect_faces(video_path, output_folder):

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)


    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        if frame_count % int(fps) == 0:
            faces = detector.detect_faces(frame)

            for i, face in enumerate(faces):
                x, y, width, height = face['box']

                face_image = frame[y:y+height, x:x+width]

                output_filename = f"{output_folder}frame_{frame_count}.jpg"
                cv2.imwrite(output_filename, face_image)
                image = Image.open(output_filename)
                new_image = image.resize((300, 300))
                new_image.save(output_filename)

                del new_image
                del face_image
            del faces
        del ret
        del frame
    del fps

    cap.release()
    del cap

# input_video_path = 'drive/MyDrive/Colab Notebooks/Final_Year_Project/test_preprocess.mp4'

import os
# os.makedirs(output_folder_path, exist_ok=True)

data= pd.read_csv("drive/MyDrive/Colab Notebooks/Final_Year_Project/FakeAVCeleb_v1.2/meta_data.csv")
data.head()
# extract_frames_and_detect_faces(input_video_path, output_folder_path)


,source,target1,target2,method,category,type,race,gender,path,file_path
0,id00076,-,-,real,A,RealVideo-RealAudio,African,men,00109.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...
1,id00166,-,-,real,A,RealVideo-RealAudio,African,men,00010.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...
2,id00173,-,-,real,A,RealVideo-RealAudio,African,men,00118.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...
3,id00366,-,-,real,A,RealVideo-RealAudio,African,men,00118.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...
4,id00391,-,-,real,A,RealVideo-RealAudio,African,men,00052.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...


In [ ]:

output_folder_path = 'drive/MyDrive/Colab Notebooks/Final_Year_Project/output_faces/'
base_path = "drive/MyDrive/Colab Notebooks/Final_Year_Project/FakeAVCeleb_v1.2/"
num_rows = len(data)
# Accessing the column data
for i in range(19525,num_rows) :
  input_video_path = base_path + data["file_path"][i] + "/" + data["path"][i]
  input_video_path = input_video_path.replace("FakeAVCeleb/", "")
  print(input_video_path, i)
  temp = data["file_path"][i]
  output_folder= f"{output_folder_path}{temp}/videoNo_{i}/"
  os.makedirs(output_folder, exist_ok=True)
  extract_frames_and_detect_faces(input_video_path, output_folder)
  del input_video_path
  del temp
  del output_folder

In [ ]:
!pip install moviepy

In [ ]:
from moviepy.editor import VideoFileClip

def extract_wav_from_video(input_video_path,output_folder,video_no):
    print(output_folder)
    input_video = VideoFileClip(input_video_path)
    audio = input_video.audio
    output_filename = f"{output_folder}videoNo_{video_no}.wav"
    # output_audio_file = os.path.join(output_folder, output_filename)
    audio.write_audiofile(output_filename, codec='pcm_s16le')
    audio.close()

In [ ]:
import os
import pandas as pd
data= pd.read_csv("drive/MyDrive/Colab Notebooks/Final_Year_Project/FakeAVCeleb_v1.2/meta_data.csv")
data.head()

In [ ]:
output_folder_path = 'drive/MyDrive/Colab Notebooks/Final_Year_Project/output_audios/'
base_path = "drive/MyDrive/Colab Notebooks/Final_Year_Project/FakeAVCeleb_v1.2/"
num_rows = len(data)
print(num_rows)
# Accessing the column data
for i in range(21565,num_rows) :
  input_video_path = base_path+data["file_path"][i]+"/"+data["path"][i]
  input_video_path = input_video_path.replace("FakeAVCeleb/", "")
  print(input_video_path, i)
  output_folder= output_folder_path + data["file_path"][i]+"/"
  os.makedirs(output_folder, exist_ok=True)
  extract_wav_from_video(input_video_path, output_folder,i)


In [ ]:
!pip install librosa

In [ ]:
import numpy as np
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
import librosa
import librosa.display
import os

In [ ]:
def create_melspectogram_from_wav(audio_file,output_folder,videoNo):
  # audio_file = "drive/MyDrive/Colab Notebooks/Final_Year_Project/output_audios/output_audio.wav"
  signal, sr = librosa.load(audio_file)
  audio_data = signal
  sample_rate = sr
  # print(audio_data, sr)

  desired_duration = 4
  if len(audio_data) > desired_duration * sample_rate:
      audio_data = audio_data[:desired_duration * sample_rate]

  n_fft = int(sample_rate * 0.025)
  hop_length = int(sample_rate * 0.010)

  mel_spectrogram = librosa.feature.melspectrogram(y=audio_data.astype(float), sr=sample_rate, n_fft=n_fft, hop_length=hop_length)
  # print(mel_spectrogram.shape)
  mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
  # print(mel_spectrogram_db.shape)
  mel_spectrogram_db = np.squeeze(mel_spectrogram_db)
  # print(mel_spectrogram_db.shape)

  plt.figure(figsize=(10, 5))
  librosa.display.specshow(mel_spectrogram_db, sr=sample_rate, cmap='viridis', fmax=8000)
  # plt.colorbar(format='%+2.0f dB')
  # plt.title('Mel Spectrogram')
  # output_folder = "drive/MyDrive/Colab Notebooks/Final_Year_Project/output_audios_mel_spectrogram"
  output_image_file = f"{output_folder}videoNo_{videoNo}.png"
  plt.savefig(output_image_file, bbox_inches='tight')
  # resizing
  image = Image.open(output_image_file)
  new_image = image.resize((300, 150))
  new_image.save(output_image_file)
  # plt.show()
  plt.close('all')
  print(f"Mel spectrogram image saved as {output_image_file} {videoNo}")

In [ ]:
output_folder_path = 'drive/MyDrive/Colab Notebooks/Final_Year_Project/output_audios_melspectogram/'
base_path = "drive/MyDrive/Colab Notebooks/Final_Year_Project/output_audios/"
num_rows = len(data)
# print(num_rows)
# Accessing the column data
for i in range(20500,num_rows) :
  temp=data["file_path"][i]
  input_video_path = f"{base_path}{temp}/videoNo_{i}.wav"
  # input_video_path = input_video_path.replace("FakeAVCeleb/", "")
  # print(input_video_path, i)
  output_folder= f"{output_folder_path}{temp}/"
  os.makedirs(output_folder, exist_ok=True)
  # print(output_folder)
  create_melspectogram_from_wav(input_video_path, output_folder,i)


In [ ]:
audio_file = "drive/MyDrive/Colab Notebooks/Final_Year_Project/output_audios/output_audio.wav"
signal, sr = librosa.load(audio_file)
audio_data = signal
sample_rate = sr
print(audio_data, sr)

desired_duration = 4
if len(audio_data) > desired_duration * sample_rate:
    audio_data = audio_data[:desired_duration * sample_rate]

In [ ]:
n_fft = int(sample_rate * 0.025)
hop_length = int(sample_rate * 0.010)

mel_spectrogram = librosa.feature.melspectrogram(y=audio_data.astype(float), sr=sample_rate, n_fft=n_fft, hop_length=hop_length)
print(mel_spectrogram.shape)
mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
print(mel_spectrogram_db.shape)
mel_spectrogram_db = np.squeeze(mel_spectrogram_db)
print(mel_spectrogram_db.shape)


In [ ]:
plt.figure(figsize=(10, 5))
librosa.display.specshow(mel_spectrogram_db, x_axis='time', y_axis='mel', sr=sample_rate, cmap='viridis', fmax=8000)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel Spectrogram')
output_folder = "drive/MyDrive/Colab Notebooks/Final_Year_Project/output_audios_mel_spectrogram"
output_image_file = os.path.join(output_folder, '1.png')
plt.savefig(output_image_file, bbox_inches='tight')

# resizing
image = Image.open(output_image_file)
new_image = image.resize((300, 300))
new_image.save(output_image_file)

plt.show()


print(f"Mel spectrogram image saved as {output_image_file}")